In [2]:
import glob
import os
import json
import re
import csv

# Define the project_info
# You would have this from your previous script's part where you parse the projects
projects = '''
Id,Name,District,Category,Cost
#1,Planting Workshops at Oerlikon,Nord,Nature,5000
#2,Footpath Gardens around Train Stations,Nord,Nature,10000
#3,Kid Festival at Leutschenpark,Nord,Culture,5000
#4,Music Studio at Kulturbahnhof Affoltern,Nord,Culture,10000
#5,Safe Bike Paths around Irchel Park,Nord,Transportation,5000
#6,More Night Buses to Oerlikon,Nord,Transportation,10000
#7,Free Open Badi Space in Wollishofen,Süd,Nature,5000
#8,A Neighborhood Garden for Wiedikon,Süd,Nature,10000
#9,Environmental Film Session for Kids,Süd,Culture,5000
#10,Car-free Sunday Festivals on Mutschellenstrasse,Süd,Culture,10000
#11,Free Bike Fixing Workshops,Süd,Transportation,5000
#12,Car Sharing System for Young People,Süd,Transportation,10000
#13,Transforming City Spaces under Trees into Gardens,Ost,Nature,5000
#14,More Trees in Bellevue & Sechseläutenplatz,Ost,Nature,10000
#15,Plant Festival in the City Centre,Ost,Culture,5000
#16,Multicultural Festival at Sechseläutenplatz,Ost,Culture,10000
#17,Bike Lanes on Seefeldstrasse,Ost,Transportation,5000
#18,Speed bumps in the City and the Lake Front,Ost,Transportation,10000
#19,Bird Houses for Zurich-Höngg,West,Nature,5000
#20,Wet Biotope as Learning Garden in Frankental,West,Nature,10000
#21,Dingtheke: Community Things Exchange Library in Wipkingen,West,Culture,5000
#22,Sustainable Cooking Workshop with Kids,West,Culture,10000
#23,Public Bicycle Moving Trailer to be Borrowed,West,Transportation,5000
#24,Car-free Langstrasse,West,Transportation,10000
'''

def get_project_info(projects):
    project_info = {}
    lines = projects.strip().split('\n')[1:]
    for line in lines:
        parts = line.split(',')
        project_id = int(parts[0][1:])
        project_info[project_id] = parts[1:]
    return project_info

project_info = get_project_info(projects)

def save_outcome_to_csv(sorted_votes, project_info, file_path):
    with open(file_path, mode='w', newline='') as csv_file:
        fieldnames = ['Rank', 'Votes', 'Id', 'Name', 'District', 'Category', 'Cost']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for rank, (project_id, total_votes) in enumerate(sorted_votes, start=1):
            project_data = project_info[project_id]
            row = {
                'Rank': rank,
                'Votes': total_votes,
                'Id': f"#{project_id}",
                'Name': project_data[0],
                'District': project_data[1],
                'Category': project_data[2],
                'Cost': project_data[3]
            }
            writer.writerow(row)

# Your updated parse_cumu_votes function
def parse_cumu_votes(response):
    point_allocations = re.findall(r'(#\d+(?:, #\d+)*)(.*?)\s*(\d+)\s*point', response, re.IGNORECASE)
    total_points = sum(int(points) for _, _, points in point_allocations)
    votes = {}
    for project_group, _, points in point_allocations:
        project_ids = re.findall(r'#(\d+)', project_group)
        normalized_points = (int(points) / total_points) * 10 if total_points != 0 else 0
        for project_id in project_ids:
            votes[int(project_id)] = normalized_points / len(project_ids)

    return votes

# Function to load votes from the JSON file
def load_votes_from_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

# Function to recalculate the scores
def recalculate_scores(votes_data):
    vote_counts = {i: 0 for i in range(1, 25)}
    for data in votes_data:
        response = data['response']
        votes = parse_cumu_votes(response)
        for project_id, points in votes.items():
            vote_counts[project_id] += points
    return vote_counts

# This function uses the existing save_outcome_to_csv function to save the corrected outcomes
def save_corrected_outcome(vote_counts, project_info, file_path):
    sorted_votes = sorted(vote_counts.items(), key=lambda x: x[1], reverse=True)
    save_outcome_to_csv(sorted_votes, project_info, file_path)

json_file = 'llama_vote/votes_llama70bq8_cumu_1.json'

# Iterate through each folder and JSON file
votes_data = load_votes_from_json(json_file)
new_vote_counts = recalculate_scores(votes_data)

# Define the new CSV file path
base_name = os.path.basename(json_file).replace('.json', '')
corrected_outcome_path = os.path.join(f'{base_name}_corr.csv')

# Save the corrected outcome to a new CSV file
save_corrected_outcome(new_vote_counts, project_info, corrected_outcome_path)

print(f"Corrected outcome saved to {corrected_outcome_path}")


Corrected outcome saved to votes_llama70bq8_cumu_1_corr.csv


In [ ]:
import json
import re

def parse_cumu_votes(response):
    point_allocations = re.findall(r'#(\d+)\s*,\s*(\d+)', response)
    votes = {}
    for project_id, points in point_allocations:
        votes[int(project_id)] = int(points)
    return votes

# Function to correct the votes data for specified agent_ids
def correct_votes_data(filename, agent_ids_to_correct):
    with open(filename, 'r') as file:
        data = json.load(file)

    for entry in data:
        if entry['agent_id'] in agent_ids_to_correct:
            print(entry)
            entry['votes'] = parse_cumu_votes(entry['response'])
            print(entry)

    new_filename = filename.split('.json')[0] + '_corrected.json'
    with open(new_filename, 'w') as file:
        json.dump(data, file, indent=4)

file_path = 'gpt4t_vote/votes_gpt4t_cumu_1.json'
agent_ids_to_correct = {4, 9, 33, 41, 56, 72, 73, 78, 83, 93, 117, 119, 123, 147, 156}

correct_votes_data(file_path, agent_ids_to_correct)


In [ ]:
import json
import csv

with open('gpt4t_vote/votes_gpt4t_cumu_1_corrected.json', 'r') as json_file:
    corrected_data = json.load(json_file)

project_votes = {}
for agent_data in corrected_data:
    for project_id, votes in agent_data['votes'].items():
        project_id_int = int(project_id)
        project_votes[project_id_int] = project_votes.get(project_id_int, 0) + votes

corrected_csv_data = []
with open('gpt4t_vote/outcome_gpt4t_cumu_1.csv', 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        project_number = int(row['Id'].replace('#', ''))
        row['Votes'] = project_votes.get(project_number, 0)
        corrected_csv_data.append(row)

with open('gpt4t_vote/outcome_gpt4t_cumu_1_corrected.csv', 'w', newline='') as csv_file:
    fieldnames = corrected_csv_data[0].keys()
    csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    csv_writer.writeheader()
    csv_writer.writerows(corrected_csv_data)

print("The CSV file has been corrected and saved as 'outcome_gpt4t_cumu_1_corrected.csv'.")
